### Parsing

##### We chose to parse the dataset from Stanford as it has more information and characteristics than the one from Kaggle. The dataset from Kaggle is a subset of the one from Stanford, so we decided to use the original one.